# Use Case 1 - Calculate Area of Land Cove

## Initialization

In [1]:
# Google Earth Initialization
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1Adeu5BU11h1fAQet9Q05Ww8Tc1uIr-rukexl-wUT2TACwMfRlg-huibIaH0



Successfully saved authorization token.


In [2]:
# Matplotlib Initialization
from simpledbf import Dbf5
from dbfread import DBF
from geopy.geocoders import Nominatim
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType, DoubleType
from pyspark.sql.functions import col, concat_ws, udf
import numpy as np
import pandas as pd
import geemap
import pprint

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


## Load Indonesia Dataset

In [20]:
#Initialize Spark Session
sparkMaster = 'local[*]'
sparkAppName = 'Py-RevalueNature-Case2'
spark = SparkSession \
    .builder \
    .master(sparkMaster) \
    .appName(sparkAppName) \
    .config('spark.jars.packages', 'com.github.sandrasi.geodata:geopy-spark_2.12:1.0.0') \
    .getOrCreate()

In [21]:
# Test Load Digimon Dataset using Spark
digimon_folderpath = 'D:\\00 Project\\00 My Project\\Dataset\\Digimon Dataset\\'
digimon_filename = 'DigiDB_digimonlist.csv'
df_digimon = spark.read.format("csv").load(digimon_folderpath+digimon_filename)
print(df_digimon.show(5))

+------+-------+-----+----+---------+------+-----------+--------+-------+--------+--------+--------+--------+
|   _c0|    _c1|  _c2| _c3|      _c4|   _c5|        _c6|     _c7|    _c8|     _c9|    _c10|    _c11|    _c12|
+------+-------+-----+----+---------+------+-----------+--------+-------+--------+--------+--------+--------+
|Number|Digimon|Stage|Type|Attribute|Memory|Equip Slots|Lv 50 HP|Lv50 SP|Lv50 Atk|Lv50 Def|Lv50 Int|Lv50 Spd|
|     1|Kuramon| Baby|Free|  Neutral|     2|          0|     590|     77|      79|      69|      68|      95|
|     2|Pabumon| Baby|Free|  Neutral|     2|          0|     950|     62|      76|      76|      69|      68|
|     3|Punimon| Baby|Free|  Neutral|     2|          0|     870|     50|      97|      87|      50|      75|
|     4|Botamon| Baby|Free|  Neutral|     2|          0|     690|     68|      77|      95|      76|      61|
+------+-------+-----+----+---------+------+-----------+--------+-------+--------+--------+--------+--------+
only showi

In [22]:
# Load Indonesia Dataset using Spark
file_path = 'D:\\00 Project\\00 My Project\\Dataset\\Revalue Nature\\Case 1\\gadm36_IDN_shp\\'
file_4 = 'gadm36_IDN_4.dbf'
records = DBF(file_path+file_4, encoding="latin1")

# define the schema of the DataFrame
schema = StructType([
    StructField("GID_0", StringType(), True),
    StructField("NAME_0", StringType(), True),
    StructField("GID_1", StringType(), True),
    StructField("NAME_1", StringType(), True),
    StructField("GID_2", StringType(), True),
    StructField("NAME_2", StringType(), True),
    StructField("GID_3", StringType(), True),
    StructField("NAME_3", StringType(), True),
    StructField("GID_4", StringType(), True),
    StructField("NAME_4", StringType(), True),
    StructField("VARNAME_4", StringType(), True),
    StructField("TYPE_4", StringType(), True),
    StructField("ENGTYPE_4", StringType(), True),
    StructField("CC_4", StringType(), True),
    # add more fields as needed
])

# create a list of dictionaries containing the data
data = [dict(record) for record in records]

# create a PySpark DataFrame from the data
df_indonesia = spark.createDataFrame(data, schema)
print(df_indonesia.show(5))

+-----+---------+-------+------+---------+----------+-----------+----------------+-------------+-----------+---------+------+---------+----------+
|GID_0|   NAME_0|  GID_1|NAME_1|    GID_2|    NAME_2|      GID_3|          NAME_3|        GID_4|     NAME_4|VARNAME_4|TYPE_4|ENGTYPE_4|      CC_4|
+-----+---------+-------+------+---------+----------+-----------+----------------+-------------+-----------+---------+------+---------+----------+
|  IDN|Indonesia|IDN.1_1|  Aceh|IDN.1.2_1|Aceh Barat|IDN.1.2.1_1|Arongan Lambalek|IDN.1.2.1.1_1| Alue Bagok|         |  Desa|  Village|1107062026|
|  IDN|Indonesia|IDN.1_1|  Aceh|IDN.1.2_1|Aceh Barat|IDN.1.2.1_1|Arongan Lambalek|IDN.1.2.1.2_1| Alue Batee|         |  Desa|  Village|1107062027|
|  IDN|Indonesia|IDN.1_1|  Aceh|IDN.1.2_1|Aceh Barat|IDN.1.2.1_1|Arongan Lambalek|IDN.1.2.1.3_1|Alue Sundak|         |  Desa|  Village|1107062025|
|  IDN|Indonesia|IDN.1_1|  Aceh|IDN.1.2_1|Aceh Barat|IDN.1.2.1_1|Arongan Lambalek|IDN.1.2.1.4_1|    Arongan|         |

In [39]:
# Initialize Nominatim API
geolocator = Nominatim(user_agent="Geo Locator")

def get_coordinate(location):
    try:
        coordinate = geolocator.geocode(location)
        return (coordinate.longitude, coordinate.latitude)
    except:
        # return {'longitude': 0, 'latitude': 0}
        return (None, None)

addr = "Indonesia,Jawa Timur,Jember,Balung"
coordinate = get_coordinate(addr)
print("The longitude of the location is: ", coordinate[0])
print("The latitude of the location is: ", coordinate[1])

The longitude of the location is:  113.54024450668652
The latitude of the location is:  -8.28236725


In [48]:
# Copy into new Dataframe
df_coordinate = df_indonesia

# Get Long and Lat
def get_long(value):
    return get_coordinate(value)[0]
def get_lati(value):
    return get_coordinate(value)[1]

# Register the UDF
geocode_long_udf = udf(get_long, DoubleType())
geocode_lati_udf = udf(get_lati, DoubleType())
# geocode_udf = udf(get_coordinate, StructType([StructField("longitude", DoubleType()), StructField("latitude", DoubleType())]))

# Apply the lambda function and create a new column
df_coordinate = df_coordinate.withColumn("location", concat_ws(",", df_coordinate["NAME_0"], df_coordinate["NAME_1"], df_coordinate["NAME_2"], df_coordinate["NAME_3"]).cast(StringType()))
df_coordinate = df_coordinate.withColumn("longitude", geocode_long_udf(df_coordinate["location"]))
df_coordinate = df_coordinate.withColumn("latitude", geocode_lati_udf(df_coordinate["location"]))

In [49]:
print(df_coordinate.printSchema())

root
 |-- GID_0: string (nullable = true)
 |-- NAME_0: string (nullable = true)
 |-- GID_1: string (nullable = true)
 |-- NAME_1: string (nullable = true)
 |-- GID_2: string (nullable = true)
 |-- NAME_2: string (nullable = true)
 |-- GID_3: string (nullable = true)
 |-- NAME_3: string (nullable = true)
 |-- GID_4: string (nullable = true)
 |-- NAME_4: string (nullable = true)
 |-- VARNAME_4: string (nullable = true)
 |-- TYPE_4: string (nullable = true)
 |-- ENGTYPE_4: string (nullable = true)
 |-- CC_4: string (nullable = true)
 |-- location: string (nullable = false)
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)

None


In [56]:
# Write dataframe into csv
df_coordinate.select("NAME_4", "location", "longitude", "latitude").write.format("csv").mode('overwrite').save("./Indonesia_Coordinate_csv")
# df_coordinate.select("NAME_4", "location").write.format("csv").mode('overwrite').save("./Indonesia_Coordinate_csv")

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 814, in main
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 650, in read_udfs
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 375, in read_single_udf
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 87, in read_command
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 174, in _read_with_length
    return self.loads(obj)
           ^^^^^^^^^^^^^^^
  File "D:\03 Data Tools\spark-3.4.1-bin-hadoop3\python\lib\pyspark.zip\pyspark\serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [55]:
df = spark.read.csv("./Indonesia_Coordinate_csv", StructType([StructField("NAME_4", StringType(), True), StructField("location", StringType(), True)]))
print(df.show(5))

+------------+--------------------+
|      NAME_4|            location|
+------------+--------------------+
|    Air Baru|Indonesia,Sumater...|
|  Bumi Genap|Indonesia,Sumater...|
|Gedung Nyawa|Indonesia,Sumater...|
| Gedung Wani|Indonesia,Sumater...|
|Karang Endah|Indonesia,Sumater...|
+------------+--------------------+
only showing top 5 rows

None


## Load Dataset Google Earth Engine

In [6]:
dataset = ee.ImageCollection('ESA/WorldCover/v200').first()
print(dataset)

# Print image object WITH call to getInfo(); prints image metadata.
pp = pprint.PrettyPrinter(indent=2 ,width=15, compact=True)
pp.pprint(dataset.getInfo())

# Print basic information about the dataset
print("Number of bands:", dataset.bandNames().length().getInfo())
print("Projection:", dataset.projection().getInfo())
print("Scale:", dataset.projection().nominalScale().getInfo())

ee.Image({
  "functionInvocationValue": {
    "functionName": "Collection.first",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue": "ESA/WorldCover/v200"
            }
          }
        }
      }
    }
  }
})
{ 'bands': [ { 'crs': 'EPSG:4326',
               'crs_transform': [ 8.333333333333333e-05,
                                  0,
                                  -180,
                                  0,
                                  -8.333333333333333e-05,
                                  84],
               'data_type': { 'max': 255,
                              'min': 0,
                              'precision': 'int',
                              'type': 'PixelType'},
               'dimensions': [ 4320000,
                               1728000],
               'id': 'Map'}],
  'id': 'ESA/WorldCover/v200/2021',
 

In [7]:
# Define the visualization parameters
visualization = {
    'bands': ['Map']
}

# Center the map on the specified coordinates and zoom level
center = [0.467, 114.193]
zoom = 5

# Display the dataset on the map using geemap
Map = geemap.Map(center=center, zoom=zoom)
Map.addLayer(dataset, visualization, 'Landcover')
data = pd.read_csv("data.csv")
Map.add_circle_markers_from_xy(data, x="longitude", y="latitude", radius=10, color="red")

print(data.head(5))
Map

   location  latitude  longitude
0  Surabaya    -7.124    112.545
1      Bali    -8.397    114.951


Map(center=[0.467, 114.193], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(child…

## Load Dataset Indonesia ShapeFile

In [28]:
file_path = 'D:\\00 Project\\00 My Project\\Dataset\\Revalue Nature\\Case 1\\gadm36_IDN_shp\\'
file_1 = 'gadm36_IDN_1.dbf'
file_2 = 'gadm36_IDN_2.dbf'
file_3 = 'gadm36_IDN_3.dbf'
file_4 = 'gadm36_IDN_4.dbf'
dbf1 = Dbf5(file_path+file_1)
df1 = dbf1.to_dataframe()
dbf2 = Dbf5(file_path+file_2)
df2 = dbf2.to_dataframe()
dbf3 = Dbf5(file_path+file_3)
df3 = dbf3.to_dataframe()
dbf4 = Dbf5(file_path+file_4)
df4 = dbf4.to_dataframe()
print(df1.shape, df2.shape, df3.shape, df4.shape)

(33, 10) (502, 13) (6696, 16) (77474, 14)


In [29]:
print(df4.dtypes)

GID_0         object
NAME_0        object
GID_1         object
NAME_1        object
GID_2         object
NAME_2        object
GID_3         object
NAME_3        object
GID_4         object
NAME_4        object
VARNAME_4    float64
TYPE_4        object
ENGTYPE_4     object
CC_4          object
dtype: object


In [30]:
# Data cleansing
df4 = df4.replace(np.nan, '', regex=True)
print(df4[(df4['NAME_2'] == 'Sungai Penuh') & (df4['NAME_4'] == 'Sumur Anyir')])
print(df4[(df4['NAME_2'] == 'Parepare') & (df4['NAME_3'] == 'Bacukiki')])

      GID_0     NAME_0    GID_1 NAME_1      GID_2        NAME_2        GID_3  \
12529   IDN  Indonesia  IDN.8_1  Jambi  IDN.8.8_1  Sungai Penuh  IDN.8.8.1_1   

      NAME_3          GID_4       NAME_4 VARNAME_4     TYPE_4  \
12529         IDN.8.8.1.1_1  Sumur Anyir            Kelurahan   

             ENGTYPE_4        CC_4  
12529  Urban Community  1572030009  
      GID_0     NAME_0     GID_1            NAME_1        GID_2    NAME_2  \
60522   IDN  Indonesia  IDN.26_1  Sulawesi Selatan  IDN.26.16_1  Parepare   
60523   IDN  Indonesia  IDN.26_1  Sulawesi Selatan  IDN.26.16_1  Parepare   
60524   IDN  Indonesia  IDN.26_1  Sulawesi Selatan  IDN.26.16_1  Parepare   
60525   IDN  Indonesia  IDN.26_1  Sulawesi Selatan  IDN.26.16_1  Parepare   

               GID_3    NAME_3            GID_4            NAME_4 VARNAME_4  \
60522  IDN.26.16.1_1  Bacukiki  IDN.26.16.1.1_1                               
60523  IDN.26.16.1_1  Bacukiki  IDN.26.16.1.2_1    Galung Maloang             
60524  IDN.

In [ ]:
# Concatenate all locations column into single column
df4['location'] = df4[["NAME_0", 'NAME_1', 'NAME_2', 'NAME_3']].apply(",".join, axis=1)
# # df4['location'] = df4[['NAME_0', 'NAME_1', 'NAME_2', 'NAME_3']].agg(', '.join, axis=1)

# For Testing purposes
# df5 = df4[(df4['NAME_2'] == 'Sungai Penuh') & (df4['NAME_4'] == 'Sumur Anyir') | ((df4['NAME_2'] == 'Parepare') & (df4['NAME_3'] == 'Bacukiki'))]
# df5 = df4[df4['NAME_2'] == 'Parepare']
df5 = df4

df5['coordinate'] = df5['location'].apply(lambda x: get_coordinate(x))
df5['longitude'] = df5['coordinate'].apply(lambda x: x[0])
df5['latitude'] = df5['coordinate'].apply(lambda x: x[1])
print(df5.head(10))